# vLLM library for LLM inference - 2024 code analysis

## LLM class for offline inference

An LLM for generating texts from given prompts and sampling parameters.

This class includes a tokenizer, a language model (possibly distributed across multiple GPUs), and GPU memory space allocated for intermediate states (aka KV cache). Given a batch of prompts and sampling parameters, this class generates texts from the model, using an intelligent batching mechanism and efficient memory management.

NOTE: This class is intended to be used for offline inference. For online serving, use the `AsyncLLMEngine` class instead.

NOTE: For the comprehensive list of arguments, see `EngineArgs`.

### __init__()

Parameters

- model: The name or path of a HuggingFace Transformers model.
- tokenizer: The name or path of a HuggingFace Transformers tokenizer.
- tokenizer_mode: The tokenizer mode. "auto" will use the fast tokenizer if available, and "slow" will always use the slow tokenizer.
- skip_tokenizer_init: If true, skip initialization of tokenizer and detokenizer. Expect valid prompt_token_ids and None for prompt from the input.
- trust_remote_code: Trust remote code (e.g., from HuggingFace) when downloading the model and tokenizer.
- tensor_parallel_size: The number of GPUs to use for distributed execution with tensor parallelism.
- dtype: The data type for the model weights and activations. Currently, we support `float32`, `float16`, and `bfloat16`. If `auto`, we use the `torch_dtype` attribute specified in the model config file. However, if the `torch_dtype` in the config is `float32`, we will use `float16` instead.
- quantization: The method used to quantize the model weights. Currently, we support "awq", "gptq", "squeezellm", and "fp8" (experimental). If None, we first check the `quantization_config` attribute in the model config file. If that is None, we assume the model weights are not quantized and use `dtype` to determine the data type of the weights.
- revision: The specific model version to use. It can be a branch name, a tag name, or a commit id.
- tokenizer_revision: The specific tokenizer version to use. It can be a branch name, a tag name, or a commit id.
- seed: The seed to initialize the random number generator for sampling.
- gpu_memory_utilization: The ratio (between 0 and 1) of GPU memory to reserve for the model weights, activations, and KV cache. Higher values will increase the KV cache size and thus improve the model's throughput. However, if the value is too high, it may cause out-of- memory (OOM) errors.
- swap_space: The size (GiB) of CPU memory per GPU to use as swap space. This can be used for temporarily storing the states of the requests when their `best_of` sampling parameters are larger than 1. If all requests will have `best_of=1`, you can safely set this to 0. Otherwise, too small values may cause out-of-memory (OOM) errors.
- enforce_eager: Whether to enforce eager execution. If True, we will disable CUDA graph and always execute the model in eager mode. If False, we will use CUDA graph and eager execution in hybrid.
- max_context_len_to_capture: Maximum context len covered by CUDA graphs. When a sequence has context length larger than this, we fall back to eager mode (DEPRECATED. Use `max_seq_len_to_capture` instead).
- max_seq_len_to_capture: Maximum sequence len covered by CUDA graphs. When a sequence has context length larger than this, we fall back to eager mode.
- disable_custom_all_reduce: See ParallelConfig
- \*\*kwargs => **see EngineArgs arguments below**

Parameters types and default values

- model: str,
- tokenizer: Optional[str] = None,
- tokenizer_mode: str = "auto",
- skip_tokenizer_init: bool = False,
- trust_remote_code: bool = False,
- tensor_parallel_size: int = 1,
- dtype: str = "auto",
- quantization: Optional[str] = None,
- revision: Optional[str] = None,
- tokenizer_revision: Optional[str] = None,
- seed: int = 0,
- gpu_memory_utilization: float = 0.9,
- swap_space: int = 4,
- enforce_eager: bool = False,
- max_context_len_to_capture: Optional[int] = None,
- max_seq_len_to_capture: int = 8192,
- disable_custom_all_reduce: bool = False,
- **kwargs,

Returns

Creates an LLM engine from the engine arguments.

- engine_args = EngineArgs(...)
- self.llm_engine = LLMEngine.from_engine_args(engine_args, usage_context=UsageContext.LLM_CLASS)
- self.request_counter = Counter()

### EngineArgs - arguments for the vLLM engine

```python
    model: str
    served_model_name: Optional[Union[List[str]]] = None
    tokenizer: Optional[str] = None
    skip_tokenizer_init: bool = False
    tokenizer_mode: str = 'auto'
    trust_remote_code: bool = False
    download_dir: Optional[str] = None
    load_format: str = 'auto'
    dtype: str = 'auto'
    kv_cache_dtype: str = 'auto'
    quantization_param_path: Optional[str] = None
    seed: int = 0
    max_model_len: Optional[int] = None
    worker_use_ray: bool = False
    distributed_executor_backend: Optional[str] = None
    pipeline_parallel_size: int = 1
    tensor_parallel_size: int = 1
    max_parallel_loading_workers: Optional[int] = None
    block_size: int = 16
    enable_prefix_caching: bool = False
    use_v2_block_manager: bool = False
    swap_space: int = 4  # GiB
    gpu_memory_utilization: float = 0.90
    max_num_batched_tokens: Optional[int] = None
    max_num_seqs: int = 256
    max_logprobs: int = 5  # OpenAI default value
    disable_log_stats: bool = False
    revision: Optional[str] = None
    code_revision: Optional[str] = None
    rope_scaling: Optional[dict] = None
    tokenizer_revision: Optional[str] = None
    quantization: Optional[str] = None
    enforce_eager: bool = False
    max_context_len_to_capture: Optional[int] = None
    max_seq_len_to_capture: int = 8192
    disable_custom_all_reduce: bool = False
    tokenizer_pool_size: int = 0
    tokenizer_pool_type: str = "ray"
    tokenizer_pool_extra_config: Optional[dict] = None
    enable_lora: bool = False
    max_loras: int = 1
    max_lora_rank: int = 16
    fully_sharded_loras: bool = False
    lora_extra_vocab_size: int = 256
    long_lora_scaling_factors: Optional[Tuple[float]] = None
    lora_dtype = 'auto'
    max_cpu_loras: Optional[int] = None
    device: str = 'auto'
    ray_workers_use_nsight: bool = False
    num_gpu_blocks_override: Optional[int] = None
    num_lookahead_slots: int = 0
    model_loader_extra_config: Optional[dict] = None

    # Related to Vision-language models such as llava
    image_input_type: Optional[str] = None
    image_token_id: Optional[int] = None
    image_input_shape: Optional[str] = None
    image_feature_size: Optional[int] = None
    scheduler_delay_factor: float = 0.0
    enable_chunked_prefill: bool = False

    guided_decoding_backend: str = 'outlines'
    # Speculative decoding configuration.
    speculative_model: Optional[str] = None
    num_speculative_tokens: Optional[int] = None
    speculative_max_model_len: Optional[int] = None
    speculative_disable_by_batch_size: Optional[int] = None
    ngram_prompt_lookup_max: Optional[int] = None
    ngram_prompt_lookup_min: Optional[int] = None
```

### generate()

Parameters types and default values

- prompts: A list of prompts to generate completions for.
- sampling_params: The sampling parameters for text generation. If None, we use the default sampling parameters. When it is a single value, it is applied to every prompt. When it is a list, the list must have the same length as the prompts and it is paired one by one with the prompt.
- prompt_token_ids: A list of token IDs for the prompts. If None, we use the tokenizer to convert the prompts to token IDs.
- use_tqdm: Whether to use tqdm to display the progress bar.
- lora_request: LoRA request to use for generation, if any.
- multi_modal_data: Multi modal data.

Default values

- prompts: Optional[Union[str, List[str]]] = None,
- sampling_params: Optional[Union[SamplingParams, List[SamplingParams]]] = None,
- prompt_token_ids: Optional[List[List[int]]] = None,
- use_tqdm: bool = True,
- lora_request: Optional[LoRARequest] = None,
- multi_modal_data: Optional[MultiModalData] = None,

Returns
            
A list of `RequestOutput` objects containing the generated completions in the same order as the input prompts.

1. requests_data = self._validate_and_prepare_requests(...)
2. for request_data in requests_data: self._add_request(**request_data)
3. return self._run_engine(use_tqdm)

## vLLM OpenAI-Compatible RESTful API server

vllm/entrypoints/openai/api_server.py

### Server arguments

- argument("--host", type=nullable_str, default=None, help="host name")
- argument("--port", type=int, default=8000, help="port number")
- argument("--uvicorn-log-level", type=str, default="info", choices=['debug', 'info', 'warning', 'error', 'critical', 'trace'], help="log level for uvicorn")
- argument("--allow-credentials", action="store_true", help="allow credentials")
- argument("--allowed-origins", type=json.loads, default=["*"], help="allowed origins")
- argument("--allowed-methods", type=json.loads, default=["*"], help="allowed methods")
- argument("--allowed-headers", type=json.loads, default=["*"], help="allowed headers")
- argument("--api-key", type=nullable_str, default=None, help="If provided, the server will require this key to be presented in the header.")
- argument("--lora-modules", type=nullable_str, default=None, nargs='+', action=LoRAParserAction, help="LoRA module configurations in the format name=path. Multiple modules can be specified.")
- argument("--chat-template", type=nullable_str, default=None, help="The file path to the chat template, or the template in single-line form for the specified model")
- argument("--response-role", type=nullable_str, default="assistant", help="The role name to return if `request.add_generation_prompt=true`.")
- argument("--ssl-keyfile", type=nullable_str, default=None, help="The file path to the SSL key file")
- argument("--ssl-certfile", type=nullable_str, default=None, help="The file path to the SSL cert file")
- argument("--ssl-ca-certs", type=nullable_str, default=None, help="The CA certificates file")
- argument( "--ssl-cert-reqs", type=int, efault=int(ssl.CERT_NONE), help="Whether client certificate is required (see stdlib ssl module's)")
- argument("--root-path", type=nullable_str, default=None, help="FastAPI root_path when app is behind a path based routing proxy")
- argument( "--middleware", type=nullable_str, action="append", default=[], help="Additional ASGI middleware to apply to the app. We accept multiple --middleware arguments. The value should be an import path. If a function is provided, vLLM will add it to the server "using @app.middleware('http'). If a class is provided, vLLM will add it to the server using app.add_middleware().")

### AsyncEngine arguments

- argument('--engine-use-ray', action='store_true', help='Use Ray to start the LLM engine in a separate process as the server process.')
- argument('--disable-log-requests', action='store_true', help='Disable logging requests.')
- argument('--max-log-len', type=int, default=None, help='Max number of prompt characters or prompt  ID numbers being printed in log. Default: Unlimited')

### Engine arguments

> Model arguments

- argument('--model', type=str, default='facebook/opt-125m', help='Name or path of the huggingface model to use.')
- argument('--tokenizer', type=nullable_str, default=EngineArgs.tokenizer, help='Name or path of the huggingface tokenizer to use.')
- argument('--skip-tokenizer-init', action='store_true', help='Skip initialization of tokenizer and detokenizer')
- argument('--revision', type=nullable_str, default=None, help='The specific model version to use. It can be a branch name, a tag name, or a commit id. If unspecified, will use the default version.')
- argument('--code-revision', type=nullable_str, default=None, help='The specific revision to use for the model code on Hugging Face Hub. It can be a branch name, a tag name, or a commit id. If unspecified, will use the default version.')
- argument('--tokenizer-revision', type=nullable_str, default=None, help='The specific tokenizer version to use. It can be a branch  name, a tag name, or a commit id. If unspecified, will use the default version.')
- argument('--tokenizer-mode', type=str, default=EngineArgs.tokenizer_mode, choices=['auto', 'slow'], help='The tokenizer mode.\n\n* "auto" will use the fast tokenizer if available.\n* "slow" will always use the slow tokenizer.')
- argument('--trust-remote-code', action='store_true', help='Trust remote code from huggingface.')
- argument('--download-dir', type=nullable_str, default=EngineArgs.download_dir, help='Directory to download and load the weights, default to the default cache dir of huggingface.')
- argument('--load-format', type=str, default=EngineArgs.load_format, choices=['auto', 'pt', 'safetensors', 'npcache', 'dummy', 'tensorizer'], help='The format of the model weights to load.\n\n * "auto" will try to load the weights in the safetensors format and fall back to the pytorch bin format if safetensors format is not available.\n * "pt" will load the weights in the pytorch bin format.\n * "safetensors" will load the weights in the safetensors format.\n * "npcache" will load the weights in pytorch format and store a numpy cache to speed up the loading.\n * "dummy" will initialize the weights with random values, which is mainly for profiling.\n * "tensorizer" will load the weights using tensorizer from CoreWeave. See the Tensorize vLLM Model script in the Examplessection for more information.\n')
- argument('--dtype', type=str, default=EngineArgs.dtype, choices=['auto', 'half', 'float16', 'bfloat16', 'float', 'float32'], help='Data type for model weights and activations.\n\n * "auto" will use FP16 precision for FP32 and FP16 models, and BF16 precision for BF16 models.\n * "half" for FP16. Recommended for AWQ quantization.\n * "float16" is the same as "half".\n * "bfloat16" for a balance between precision and range.\n * "float" is shorthand for FP32 precision.\n * "float32" for FP32 precision.')
- argument('--max-model-len', type=int, default=EngineArgs.max_model_len, help='Model context length. If unspecified, will be automatically derived from the model config.')
- argument('--rope-scaling', default=None, type=json.loads, help='RoPE scaling configuration in JSON format. For example, {"type":"dynamic","factor":2.0}')
- argument('--model-loader-extra-config', type=nullable_str, default=EngineArgs.model_loader_extra_config, help='Extra config for model loader. This will be passed to the model loader corresponding to the chosen load_format. This should be a JSON string that will be parsed into a dictionary.')
- argument("--served-model-name", nargs="+", type=str, default=None, help="The model name(s) used in the API. If multiple names are provided, the server will respond to any of the provided names. The model name in the model field of a response will be the first name in this list. If not specified, the model name will be the same as the `--model` argument. Noted that this name(s) will also be used in `model_name` tag content of prometheus metrics, if multiple names provided, metricstag will take the first one.")

> Guided decoding

- argument('--guided-decoding-backend', type=str, default='outlines', choices=['outlines', 'lm-format-enforcer'], help='Which engine will be used for guided decoding (JSON schema / regex etc) by default. Currently support https://github.com/outlines-dev/outlines and https://github.com/noamgat/lm-format-enforcer. Can be overridden per request via guided_decoding_backend parameter.')

> Parallel arguments

- argument('--distributed-executor-backend', choices=['ray', 'mp'], default=EngineArgs.distributed_executor_backend, help='Backend to use for distributed serving. When more than 1 GPU is used, will be automatically set to "ray" if installed or "mp" (multiprocessing) otherwise.')
- argument('--worker-use-ray', action='store_true', help='Deprecated, use --distributed-executor-backend=ray.')
- argument('--pipeline-parallel-size', '-pp', type=int, default=EngineArgs.pipeline_parallel_size, help='Number of pipeline stages.')
- argument('--tensor-parallel-size', '-tp', type=int, default=EngineArgs.tensor_parallel_size, help='Number of tensor parallel replicas.')
- argument('--max-parallel-loading-workers', type=int, default=EngineArgs.max_parallel_loading_workers, help='Load model sequentially in multiple batches, to avoid RAM OOM when using tensor parallel and large models.')
- argument('--ray-workers-use-nsight', action='store_true', help='If specified, use nsight to profile Ray workers.')
- argument('--disable-custom-all-reduce', action='store_true', default=EngineArgs.disable_custom_all_reduce, help='See ParallelConfig.')

> CPU / GPU optimization

- argument("--device", type=str, default=EngineArgs.device, choices=["auto", "cuda", "neuron", "cpu"], help='Device type for vLLM execution.')
- argument('--gpu-memory-utilization', type=float, default=EngineArgs.gpu_memory_utilization, help='The fraction of GPU memory to be used for the model executor, which can range from 0 to 1. For example, a value of 0.5 would imply 50%% GPU memory utilization. If unspecified, will use the default value of 0.9.')
- argument('--num-gpu-blocks-override', type=int, default=None, help='If specified, ignore GPU profiling result and use this number of GPU blocks. Used for testing preemption.')
- argument('--enforce-eager', action='store_true', help='Always use eager-mode PyTorch. If False, will use eager mode and CUDA graph in hybrid for maximal performance and flexibility.')
- argument('--max-seq-len-to-capture', type=int, default=EngineArgs.max_seq_len_to_capture, help='Maximum sequence length covered by CUDA graphs. When a sequence has context length larger than this, we fall back to eager mode.')
- argument('--swap-space', type=int, default=EngineArgs.swap_space, help='CPU swap space size (GiB) per GPU.')
- argument('--seed', type=int, default=EngineArgs.seed, help='Random seed for operations.')

> Quantization settings.

- argument('--quantization', '-q', type=nullable_str, choices=[*QUANTIZATION_METHODS, None], default=EngineArgs.quantization, help='Method used to quantize the weights. If None, we first check the `quantization_config` attribute in the model config file. If that is None, we assume the model weights are not quantized and use `dtype` to determine the data type of the weights.')
- argument('--kv-cache-dtype', type=str, choices=['auto', 'fp8', 'fp8_e5m2', 'fp8_e4m3'], default=EngineArgs.kv_cache_dtype, help='Data type for kv cache storage. If "auto", will use model  data type. CUDA 11.8+ supports fp8 (=fp8_e4m3) and fp8_e5m2. ROCm (AMD GPU) supports fp8 (=fp8_e4m3)')
- argument('--quantization-param-path' type=nullable_str, default=None, help='Path to the JSON file containing the KV cache scaling factors. This should generally be supplied, when 'KV cache dtype is FP8. Otherwise, KV cache scaling factors default to 1.0, which may cause accuracy issues. FP8_E5M2 (without scaling) is only supported on cuda version greater than 11.8. On ROCm (AMD GPU), FP8_E4M3 is instead supported for common inference criteria.')

> Speculative decoding

- argument('--speculative-model', type=nullable_str, default=EngineArgs.speculative_model, help='The name of the draft model to be used in speculative decoding.')
- argument('--num-speculative-tokens', type=int, default=EngineArgs.num_speculative_tokens, help='The number of speculative tokens to sample from the draft model in speculative decoding.')
- argument('--speculative-max-model-len', type=int, default=EngineArgs.speculative_max_model_len, help='The maximum sequence length supported by the draft model. Sequences over this length will skip speculation.')
- argument('--speculative-disable-by-batch-size', type=int, default=EngineArgs.speculative_disable_by_batch_size, help='Disable speculative decoding for new incoming requests if the number of enqueue requests is larger than this value.')
- argument('--ngram-prompt-lookup-max', type=int, default=EngineArgs.ngram_prompt_lookup_max, help='Max size of window for ngram prompt lookup in speculative decoding.')
- argument('--ngram-prompt-lookup-min', type=int, default=EngineArgs.ngram_prompt_lookup_min, help='Min size of window for ngram prompt lookup in speculative decoding.')

> LoRA related configs

- argument('--enable-lora', action='store_true', help='If True, enable handling of LoRA adapters.')
- argument('--max-loras', type=int, default=EngineArgs.max_loras, help='Max number of LoRAs in a single batch.')
- argument('--max-lora-rank', type=int, default=EngineArgs.max_lora_rank, help='Max LoRA rank.')
- argument('--lora-extra-vocab-size', type=int, default=EngineArgs.lora_extra_vocab_size, help=('Maximum size of extra vocabulary that can be present in a LoRA adapter (added to the base model vocabulary).'))
- argument('--lora-dtype', type=str, default=EngineArgs.lora_dtype, choices=['auto', 'float16', 'bfloat16', 'float32'], help=('Data type for LoRA. If auto, will default to base model dtype.'))
- argument('--long-lora-scaling-factors', type=nullable_str, default=EngineArgs.long_lora_scaling_factors, help=('Specify multiple scaling factors (which can be different from base model scaling factor - see eg. Long LoRA) to allow for multiple LoRA adapters trained with those scaling factors to be used at the same time. If not specified, only adapters trained with the base model scaling factor are allowed.'))
- argument('--max-cpu-loras', type=int, default=EngineArgs.max_cpu_loras, help=('Maximum number of LoRAs to store in CPU memory. Must be >= than max_num_seqs. Defaults to max_num_seqs.'))
- argument('--fully-sharded-loras', action='store_true', help=('By default, only half of the LoRA computation is sharded with tensor parallelism. Enabling this will use the fully sharded layers. At high sequence length, max rank or tensor parallel size, this is likely faster.'))

> Scheduler, batches, pools, stats

- argument('--scheduler-delay-factor', type=float, default=EngineArgs.scheduler_delay_factor, help='Apply a delay (of delay factor multiplied by previous prompt latency) before scheduling next prompt.')
- argument('--enable-chunked-prefill', action='store_true', help='If set, the prefill requests can be chunked based on the max_num_batched_tokens.')
- argument('--max-num-batched-tokens', type=int, default=EngineArgs.max_num_batched_tokens, help='Maximum number of batched tokens per iteration.')
- argument('--max-num-seqs', type=int, default=EngineArgs.max_num_seqs, help='Maximum number of sequences per iteration.')
- argument('--max-logprobs', type=int, default=EngineArgs.max_logprobs, help=('Max number of log probs to return logprobs is specified in SamplingParams.'))
- argument('--tokenizer-pool-size', type=int, default=EngineArgs.tokenizer_pool_size, help='Size of tokenizer pool to use for asynchronous tokenization. If 0, will use synchronous tokenization.')
- argument('--tokenizer-pool-type', type=str, default=EngineArgs.tokenizer_pool_type, help='Type of tokenizer pool to use for asynchronous tokenization. Ignored if tokenizer_pool_size is 0.')
- argument('--tokenizer-pool-extra-config', type=nullable_str, default=EngineArgs.tokenizer_pool_extra_config, help='Extra config for tokenizer pool. This should be a JSON string that will be parsed into a dictionary. Ignored if tokenizer_pool_size is 0.')
- argument('--disable-log-stats', action='store_true', help='Disable logging statistics.')

> KV cache arguments

- argument('--block-size', type=int, default=EngineArgs.block_size, choices=[8, 16, 32], help='Token block size for contiguous chunks of tokens.')
- argument('--enable-prefix-caching', action='store_true', help='Enables automatic prefix caching.')
- argument('--use-v2-block-manager', action='store_true', help='Use BlockSpaceMangerV2.')
- argument('--num-lookahead-slots', type=int, default=EngineArgs.num_lookahead_slots, help='Experimental scheduling config necessary for speculative decoding. This will be replaced by speculative config in the future; it is present to enable correctness tests until then.')

> Related to Vision-language models such as llava

- argument('--image-input-type', type=nullable_str, default=None, choices=[t.name.lower() for t in VisionLanguageConfig.ImageInputType], help=('The image input type passed into vLLM. Should be one of "pixel_values" or "image_features".'))
- argument('--image-token-id',type=int, default=None, help=('Input id for image token.'))
- argument('--image-input-shape', type=nullable_str, default=None, help=('The biggest image input shape (worst for memory footprint) given an input type. Only used for vLLM\'s profile_run.'))
- argument('--image-feature-size', type=int, default=None, help=('The image feature size along the context dimension.'))

## LLMEngine class - that receives requests and generates texts

This is the main class for the vLLM engine. It receives requests
from clients and generates texts from the LLM. It includes a tokenizer, a
language model (possibly distributed across multiple GPUs), and GPU memory
space allocated for intermediate states (aka KV cache). This class utilizes
iteration-level scheduling and efficient memory management to maximize the
serving throughput.

The `LLM` class wraps this class for offline batched inference and the
`AsyncLLMEngine` class wraps this class for online serving.

NOTE: The config arguments are derived from the `EngineArgs` class. For the
comprehensive list of arguments, see `EngineArgs`.

### from_engine_args()
    
Creates an LLM engine from the engine arguments.

- engine_args: EngineArgs,
- usage_context: UsageContext = UsageContext.ENGINE_CONTEXT,
        
1. Create the engine configs.
2. Initialize the cluster and specify the executor class.
3. Create the LLM engine.

### __init__()

Parameters

- model_config: The configuration related to the LLM model.
- cache_config: The configuration related to the KV cache memory management.
- parallel_config: The configuration related to distributed execution.
- scheduler_config: The configuration related to the request scheduler.
- device_config: The configuration related to the device.
- lora_config (Optional): The configuration related to serving multi-LoRA.
- vision_language_config (Optional): The configuration related to vision language models.
- speculative_config (Optional): The configuration related to speculative decoding.
- executor_class: The model executor class for managing distributed execution.
- log_stats: Whether to log statistics.
- usage_context: Specified entry point, used for usage info collection

Parameters types and default values

- model_config: ModelConfig,
- cache_config: CacheConfig,
- parallel_config: ParallelConfig,
- scheduler_config: SchedulerConfig,
- device_config: DeviceConfig,
- load_config: LoadConfig,
- lora_config: Optional[LoRAConfig],
- vision_language_config: Optional[VisionLanguageConfig],
- speculative_config: Optional[SpeculativeConfig],
- decoding_config: Optional[DecodingConfig],
- executor_class: Type[ExecutorBase],
- log_stats: bool,
- usage_context: UsageContext = UsageContext.ENGINE_CONTEXT

## AsyncLLMEngine class - an asynchronous wrapper for LLMEngine

This class is used to wrap the LLMEngine class to make it asynchronous. It
uses asyncio to create a background loop that keeps processing incoming
requests. The LLMEngine is kicked by the generate method when there
are requests in the waiting queue. The generate method yields the outputs
from the LLMEngine to the caller.

NOTE: For the comprehensive list of arguments, see `LLMEngine`.

### __init__()

- worker_use_ray: Whether to use Ray for model workers. Required for distributed execution. Should be the same as `parallel_config.worker_use_ray`.
- engine_use_ray: Whether to make LLMEngine a Ray actor. If so, the async frontend will be executed in a separate process as the model workers.
- log_requests: Whether to log the requests.
- max_log_len: Maximum number of prompt characters or prompt ID numbers being printed in log.
- start_engine_loop: If True, the background task to run the engine will be automatically started in the generate call.
- *args: Arguments for LLMEngine.
- *kwargs: Arguments for LLMEngine.

Types 

- worker_use_ray: bool,
- engine_use_ray: bool,
- log_requests: bool = True,
- max_log_len: Optional[int] = None,
- start_engine_loop: bool = True,